In [4]:
import torch.nn as nn
import torch
import time
import sys

In [18]:
class SimpleFFNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size=1):
        super(SimpleFFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        h1 = torch.relu(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        return torch.sigmoid(self.fc3(h2))
    
    def num_parameters(self):
        total = 0
        for p in self.parameters():
            total += p.numel()
        return total

In [5]:
from torch.utils.data import Dataset, DataLoader

class RandomDataSet(Dataset):
  def __init__(self, N, f):
    R_N_f = torch.rand(N,f)
    self.X = torch.bernoulli(R_N_f)
    R_N_1 = torch.rand(N,1)
    self.Y = torch.bernoulli(R_N_1)
    self.num_features = f

  # Debemos definir __len__ para retornar el tamaño del dataset
  def __len__(self):
    return self.X.size()[0]

  # Debemos definir __getitem__ para retornar el i-ésimo
  # ejemplo en nuestro dataset.
  def __getitem__(self, i):
    return self.X[i], self.Y[i]

In [20]:
def loop_FFNN(dataset, batch_size, d1, d2, lr,
                 epochs, run_in_GPU=True, reports_every=1,
                 cheq_grad=False, init_v=1):
    
    device = 'cuda' if run_in_GPU else 'cpu'

    # d0 es la cantidad de features
    d0 = dataset.num_features

    # Crea la red
    red = SimpleFFNN(d0, d1, d2)

    # Pasa la red al dispositivo elegido
    red.to(device)

    # Muestra la cantidad de parámetros
    print('Cantidad de parámetros:', red.num_parameters())

    # Crea un dataloader desde el dataset
    data = DataLoader(dataset, batch_size, shuffle=True)

    # Crea un optimizador para el descenso de gradiente
    optimizador = torch.optim.SGD(red.parameters(), lr)

    # Define una pérdida
    perdida = torch.nn.BCELoss()

    # Comienza el entrenamiento
    tiempo_epochs = 0
    for e in range(1, epochs+1):
        inicio_epoch = time.time()

        for (x, y) in data:
            # Asegura de pasarlos a la GPU si fuera necesario
            x, y = x.to(device), y.to(device)

            # Computa la pasada hacia adelante (forward)
            y_pred = red.forward(x)

            # Computa la función de pérdida
            L = perdida(y_pred,y)

            # Computa los gradientes hacia atrás (backpropagation)
            L.backward()

            # Descenso de gradiente para actualizar los pesos
            optimizador.step()

            # Limpia los gradientes
            optimizador.zero_grad()

        tiempo_epochs += time.time() - inicio_epoch

        if e % reports_every == 0:

            # calcula la certeza de las predicciones sobre todo el conjunto
            X = dataset.X.to(device)
            Y = dataset.Y.to(device)

            # Predice usando la red
            Y_PRED = red.forward(X)

            # Predice la pérdida de todo el conjunto
            L_total = perdida(Y_PRED, Y)

            # Elige una clase dependiendo del valor de Y_PRED
            Y_PRED_BIN = (Y_PRED >= 0.5).float()

            correctos = torch.sum(Y_PRED_BIN == Y).item()

            acc = (correctos/N) * 100

            sys.stdout.write(
            '\rEpoch:{0:03d}'.format(e) + ' Acc:{0:.2f}%'.format(acc)
            + ' Loss:{0:.4f}'.format(L_total)
            + ' Tiempo/epoch:{0:.3f}s'.format(tiempo_epochs/e))



In [21]:
N = 5000 # numero de ejemplos
f = 300 # numero de features

dataset = RandomDataSet(N,f)

epochs = 50
loop_FFNN(dataset, batch_size=10, d1=300, d2=400, epochs=epochs,
             run_in_GPU=True, lr=0.05)

Cantidad de parámetros: 211101
Epoch:050 Acc:100.00% Loss:0.0001 Tiempo/epoch:0.420s